In [11]:
import os

os.environ["CONFIG_APP_DIR"] = "config"

In [12]:
import sys
import os
import pickle
os.chdir("/app")
from omegaconf import OmegaConf
from etl_pipeline.config import columns_namespace, pipeline_config
from service.proto.etl_pipeline_pb2 import Alert, Match
from etl_pipeline.config import columns_namespace
from custom.ms.datatypes.field import InputRecordField
from etl_pipeline.custom.ms.payload_loader import PayloadLoader
from etl_pipeline.custom.ms.transformations import (
    create_agent_input_agg_col_config,
    prepend_agent_name_to_ap_or_wl_or_aliases_key,
)
from etl_pipeline.custom.ms.watchlist_extractor import WatchlistExtractor
from etl_pipeline.pipeline import ETLPipeline
from pipelines.ms.json_pipeline import MSPipeline

import json
from etl_pipeline.data_processor_engine.json_engine.json_engine import JsonProcessingEngine


In [13]:
def load_alert():
    with open("API/xml_alignment_payload.json", "r") as f:
        text = json.load(f)
        match1 = Match(match_id="0", match_name="1")
        match2 = Match(match_id="1", match_name="2")
        alert = Alert(batch_id="1", alert_name="2", matches=[match1, match2])
        for key, value in text.items():
            alert.flat_payload[str(key)] = str(value)
    return alert

In [14]:
alert = load_alert()

In [15]:
payload = load_alert()
payload = payload.flat_payload

In [16]:
payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)
payload_json['match_ids'] = [int(i.match_id) for i in alert.matches]

In [17]:
payload_json['alert'].keys()

dict_keys(['headerInfo', 'inputRecordHist', 'matchRecords', 'supplementalInfo'])

In [18]:
engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline(engine, config=pipeline_config)

In [19]:

payload = pipeline.transform_standardized_to_cleansed(payload_json)
payload = pipeline.transform_cleansed_to_application(payload)

### payload

In [20]:
for match in payload["alert"]["matchRecords"]:
    print("++++++++")
    print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
    print(match["ap_all_nationalities_aggregated"], match["wl_all_nationalities_aggregated"])
    print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])

++++++++
[['', '10/10/1969']] [[]]
[None] [[[], [], [], None, None]]
[None] [None]
++++++++
[['', '10/10/1969']] [['01/11/1924']]
[None] [[['CHIC'], [], [], None, None]]
[None] [None]


In [ ]:
# new_json listy 